In [10]:
######## Create, Train, and Predict Models

# Loads train, test, and val datasets
# Instantiates Models
# Trains Models
# Predicts using Models

Import All Required Modules

In [22]:
import sys

# Manually add the project root to sys.path
sys.path.append('/Users/joaquinuriarte/Documents/GitHub/sports-betting/')

# === STEP 0: Imports
from modules.model_manager.trainer.trainer import Trainer
from modules.model_manager.predictor.predictor import Predictor
from modules.model_manager.factories.model_factory import ModelFactory
from modules.model_manager.helpers.configuration_loader import ConfigurationLoader as MMConfigLoader
from modules.model_manager.model_manager import ModelManager
from sklearn.model_selection import KFold
from modules.data_structures.model_dataset import ModelDataset
from model_binaries.utils.binary_utils import save_entity, load_entity

In [19]:
# === STEP 1: Dependency Instantiations And Global Variable Declarations

## === MODEL MANAGER
yaml_path0 = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/configs/model_v0/model_v0_000.yaml'
yaml_paths = [yaml_path0]


checkpoint = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/models'
trainer = Trainer(checkpoint)
predictor = Predictor()
model_factory = ModelFactory()
mm_configuration_loader = MMConfigLoader()

Load Train into Memory

In [16]:
# File path to load train, test, and val datasets
train_test_val_folder_path = "/Users/joaquinuriarte/Documents/GitHub/sports-betting/processed_datasets/model_v0/scaler"

In [17]:
train_dataset = load_entity(train_test_val_folder_path, "scaled_train.pkl")

Create and Train Models Using KFold Cross Validation

In [21]:
# === STEP 4: MODEL MANAGER
yamls = [yaml_path0]
model_manager = ModelManager(trainer, predictor, model_factory, mm_configuration_loader)

In [29]:
n_splits = 2
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

fold_results = []

# We'll split over the indices of train_dataset.examples
num_examples = len(train_dataset.examples)
indices = range(num_examples)

for train_index, val_index in kf.split(indices):
    # Create list of examples for each fold
    train_examples = [train_dataset.examples[i] for i in train_index]
    val_examples   = [train_dataset.examples[i] for i in val_index]
    
    # Wrap them again as ModelDataset objects
    train_fold = ModelDataset(train_examples)
    val_fold   = ModelDataset(val_examples)
    
    # Create model
    models = model_manager.create_models(yamls)

    # Train on this fold’s train/val
    metrics = model_manager.train(
        models, 
        [(train_fold, val_fold)], 
        save_after_training=True
    )

    fold_results.append(metrics)

# Aggregate fold_results (average, standard deviation, etc.)

TypeError: only integer scalar arrays can be converted to a scalar index

In [26]:
### Tenemos que set metrics infra y considerar meterlo en tensor board and return it pa poder hacer esto bien. 
### si, return for cross val purposes and consider adding tags to tensorboard logs pa que puedas verlos ahi si quieres

### remember que desp de kfold pa tune hyper params entrenas chosen model en entire train dataset and then predict on test pa get real accuracies. Maybe esto ya entonces es otro file

Get Averaged Metrics for KFold

In [27]:
# Average Kfold Metrics

Launch Tensorboard

In [10]:
%tensorboard --logdir logs/fit/a141223cdff8e65162f8bbc75c6c3d96

UsageError: Line magic function `%tensorboard` not found.
